In [17]:
target = "P01375"
# panel = "Cardiometabolic_II"

In [18]:
# Everything you need to change is in this cell.
# Set target equal to the target you want to graph (target name must be in quotes).
# Each of the paths must be filled with the locations of each of the documents on your desktop.

PLATE_LAYOUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Plate Layout.xlsx"
RAW_DATA_FILE = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\LCSET_28343_10-4-2023_EXTENDED_NPX_2023-10-20.csv"
MANIFEST_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Shah (BWH)_OLINK_SK-5T4V_2023-06-27_MN UPDATES 82823.xlsx"

In [19]:
import matplotlib.pyplot as plt
import pandas as pd

In [20]:
CSF_SAMPLE_NUMBERS = ["CSF1", "CSF2", "CSF3", "CSF4", "CSF5", "CSF6", "CSF7", "CSF8"]
UNIMPORTANT_COLUMNS = [
    "Index",
    "OlinkID",
    "Panel_Lot_Nr",
    "PlateID",
    "WellID",
    "Normalization",
    "Processing_StartDate",
    "Processing_EndDate",
    "AnalyzerID",
    "ExploreVersion",
]
X_AXIS_ORDER = [
    "Fract 6",
    "Fract 7",
    "Fract 8",
    "Fract 9",
    "Fract 10",
    "Fract 11",
    "Fract 12",
    "Fract 13",
    "Fract 14",
    "Fract 15",
]

In [23]:
data

,SampleID,Sample_Type,UniProt,Assay,MissingFreq,Panel,QC_Warning,LOD,NPX,Assay_Warning,IntraCV,InterCV,CSF_sample,Sample,Patient_ID,Health,Linear NPX
0,SM-N8JPG,SAMPLE,INC1,Incubation control 1,NaN,Cardiometabolic,PASS,NaN,0.1038,PASS,NaN,NaN,CSF1,SEC Fract 6,8351.0,Healthy,1.074600
1,SM-N8JPM,SAMPLE,INC1,Incubation control 1,NaN,Cardiometabolic,PASS,NaN,0.1803,PASS,NaN,NaN,CSF1,SEC Fract 12,8351.0,Healthy,1.133119
2,SM-N8JPH,SAMPLE,INC1,Incubation control 1,NaN,Cardiometabolic,PASS,NaN,0.0820,PASS,NaN,NaN,CSF1,SEC Fract 7,8351.0,Healthy,1.058484
3,SM-N8JPN,SAMPLE,INC1,Incubation control 1,NaN,Cardiometabolic,PASS,NaN,0.0408,PASS,NaN,NaN,CSF1,SEC Fract 13,8351.0,Healthy,1.028684
4,SM-N8JPI,SAMPLE,INC1,Incubation control 1,NaN,Cardiometabolic,PASS,NaN,-0.0992,PASS,NaN,NaN,CSF1,SEC Fract 8,8351.0,Healthy,0.933551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288763,SM-N8JS4,SAMPLE,Q96R05,RBP7,0.8977,Oncology_II,PASS,1.4381,-0.8236,PASS,1.7756,1.7756,CSF8,SEC Fract 15,8781.0,MCI,0.565030
288764,SM-N8JRY,SAMPLE,Q96R05,RBP7,0.8977,Oncology_II,PASS,1.4381,-0.9589,PASS,1.7756,1.7756,CSF8,SEC Fract 10,8781.0,MCI,0.514449
288765,SM-N8JS5,SAMPLE,Q96R05,RBP7,0.8977,Oncology_II,PASS,1.4381,6.3326,PASS,1.7756,1.7756,Plasma D Internal EV,Plasma D Internal EV,8712.0,AD,80.593970
288766,SM-N8JRZ,SAMPLE,Q96R05,RBP7,0.8977,Oncology_II,PASS,1.4381,0.5168,PASS,1.7756,1.7756,CSF8,SEC Fract 11,8781.0,MCI,1.430778


In [30]:
# clean up the raw olink data file and map the sample locations to their barcodes

plate_layout = pd.read_excel(PLATE_LAYOUT_PATH, index_col=0)
raw_data = pd.read_csv(RAW_DATA_FILE, sep=";")
raw_data = raw_data.drop(UNIMPORTANT_COLUMNS, axis=1)

data = pd.merge(raw_data, plate_layout, how="left", on="SampleID")

data["Linear NPX"] = data["NPX"].map(lambda x: 2**x)


def make_linear(x):
    return 2**x


raw_data["Linear NPX"] = raw_data["NPX"].map(make_linear)
# raw_data["Linear LOD"] = raw_data["LOD"].map(make_linear)

filtered_npx = raw_data[
    (raw_data["NPX"] > raw_data["LOD"])
    & (raw_data["QC_Warning"] == "PASS")
    & (raw_data["Assay_Warning"] == "PASS")
]


def plot_by_target(target):
    target_df = filtered_npx[
        (filtered_npx["UniProt"] == target)
        & (filtered_npx["Sample"].str.contains("Fract"))
        # & (filtered_npx["Panel"] == panel)
    ]
    medians = target_df.groupby("Sample")["Linear NPX"].median()
    medians.index = medians.index.map(lambda x: int(x.split()[-1]))
    medians = medians.sort_index()
    plt.bar(medians.index, medians, color="k", alpha=0.75)
    plt.xlabel("Fraction")
    plt.ylabel("Linear NPX")
    plt.title(f"{target} Median Fractionation Pattern")
    # plt.axhline(y = fract_df["LOD"][0], color = "red", linestyle = "--", label = "LOD")
    plt.xticks(rotation=45, ha="right")
    plt.show()
    # for sample in CSF_SAMPLE_NUMBERS:
    #     sample_df = target_df[target_df["CSF_sample"] == sample]
    #     sample_grouped = sample_df.groupby("Sample")["Linear NPX"].median()
    #     sample_grouped.index = sample_grouped.index.map(lambda x: int(x.split()[-1]))
    #     sample_grouped = sample_grouped.sort_index()
    #     plt.bar(sample_grouped.index, sample_grouped, color="k", alpha=0.75)
    #     plt.xlabel("Fraction")
    #     plt.ylabel("Linear NPX")
    #     plt.title(f"{sample} {target} Fractionation Pattern")
    #     plt.xticks(rotation=45, ha="right")
    #     plt.show()
    # healthy_target_df = filtered_npx[
    #     (filtered_npx["UniProt"] == target)
    #     # & (filtered_npx["Panel"] == panel)
    #     & (filtered_npx["Sample"].str.contains("Fract"))
    #     & (filtered_npx["Health"] == "Healthy")
    # ]

    # medians = healthy_target_df.groupby("Sample Description")["Linear NPX"].median()
    # medians.index = medians.index.map(lambda x: int(x.split()[-1]))
    # medians = medians.sort_index()
    # plt.bar(medians.index, medians, color="k", alpha=0.75)
    # plt.xlabel("Fraction")
    # plt.ylabel("Linear NPX")
    # plt.title(f"Healthy Samples {target} Median Fractionation Pattern")
    # plt.xticks(rotation=45, ha="right")
    # plt.show()

    # ad_target_df = filtered_npx[
    #     (filtered_npx["UniProt"] == target)
    #     # & (filtered_npx["Panel"] == panel)
    #     & (filtered_npx["Sample Description"].str.contains("Fract"))
    #     & (filtered_npx["Additional Information"].str.contains("AD"))
    # ]
    # medians = ad_target_df.groupby("Sample Description")["Linear NPX"].median()
    # medians.index = medians.index.map(lambda x: int(x.split()[-1]))
    # medians = medians.sort_index()
    # plt.bar(medians.index, medians, color="k", alpha=0.75)
    # plt.xlabel("Fraction")
    # plt.ylabel("Linear NPX")
    # plt.title(f"AD Samples {target} Median Fractionation Pattern")
    # plt.xticks(rotation=45, ha="right")
    # plt.show()

In [31]:
plot_by_target(target)

KeyError: 'Sample'